In [ ]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd

In [ ]:

def get_dayly_data(symbol):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    day_df = ticker.history(period=f"80d", interval="1d")
    
    # Reset the index and convert the 'Date' column to date format
    day_df.reset_index(inplace=True)
    day_df['Date'] = day_df['Date'].dt.date
    
    # Calculate technical indicators
    day_df['Day_10EMA'] = trend.EMAIndicator(day_df['Close'], window=10).ema_indicator()
    day_df['Day_RSI'] = momentum.RSIIndicator(day_df['Close'], window=3).rsi()
    day_df['Day_RSI_EMA'] = trend.EMAIndicator(day_df['Day_RSI'], window=6).ema_indicator()

    # -----------ADD EXTRA INDICATORS HERE --------------

    day_df['Day_Trend'] = np.select(
    [day_df['Day_10EMA'] > day_df['Day_10EMA'].shift(1),
     day_df['Day_10EMA'] < day_df['Day_10EMA'].shift(1)],
    ['Uptrend', 'Downtrend'],
    'Sideways')
    
    # Rename columns and drop unnecessary columns
    day_df.rename(columns={'Open': 'Day_Open', 'High': 'Day_High', 'Low': 'Day_Low', 'Close': 'Day_Close'}, inplace=True)
    day_df.drop(columns=['Volume', 'Dividends', 'Stock Splits'], inplace=True)
    day_df.dropna(inplace=True)

    # ------------------------------------ 5 MIN TIME FRAME -----------------------------------------------------------
    previes_days = 10
    df = ticker.history(period=f"{previes_days}d", interval="5m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['8EMA'] = trend.EMAIndicator(df['Close'], window=8).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['10EMA'] = trend.EMAIndicator(df['Close'], window=10).ema_indicator()

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=3).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=6).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=20)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["20EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["20EMA"]) & (df["5EMA"].shift(1) <= df["20EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["20EMA"]) & (df["5EMA"].shift(1) >= df["20EMA"].shift(1)), "Trend"] = "Downtrend"

    first_candle_high = df.groupby('Date')['High'].first()
    first_candle_low = df.groupby('Date')['Low'].first()
    
    # Create a new column for the first candle's high and low
    df['First_Candle_High'] = df['Date'].map(first_candle_high)
    df['First_Candle_Low'] = df['Date'].map(first_candle_low)

    # ------------------------------------ MERGE DATA FRAME -----------------------------------------------------------
    
    merged_df = day_df.merge(df, on='Date', how='inner')
    merged_df.dropna(inplace=True)
    
    return merged_df

data = get_dayly_data("^NSEI")

# CE SIDE TRED

In [ ]:
CE_HOLDING = False
ce_current_position = 0
ce_inisal_amount = 0
CE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Symbol','Entry Trend','Exit Trend'])

PE_HOLDING = False
pe_current_position = 0
pe_inisal_amount = 0
PE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Symbol','Entry Trend','Exit Trend'])


# CE LOGIC 

In [ ]:
def CE_ENTRY(row):
    ema_crossover_check = row['8EMA'] > row['15EMA'] and row['RSI_EMA'] <= 50
    if ema_crossover_check:
        return True
    else:
        False

def CE_EXIT(row):
    ema_crossover_check = row['8EMA'] < row['15EMA'] and row['RSI_EMA'] >= 50
    if ema_crossover_check :
        return True
    else:
        return False


# PE LOGIC

In [ ]:
def PE_ENTRY(row):
    ema_crossover_check = row['8EMA'] < row['15EMA'] and  row['RSI_EMA'] >= 50
    if ema_crossover_check:
        return True
    else:
        False

def PE_EXIT(row):
    ema_crossover_check = row['8EMA'] > row['15EMA'] or row['RSI_EMA'] <= 30
    if ema_crossover_check :
        return True
    else:
        return False

## STRETEGY RUN

In [ ]:

def Main_Stetagy_Run(row):
    global CE_HOLDING
    global ce_current_position
    global ce_inisal_amount
    global CE_data
    
    global PE_HOLDING
    global pe_current_position
    global pe_inisal_amount
    global PE_data
# ----------------------------------------------[ CE ]----------------------------------------------------
    
    if CE_ENTRY(row):
        if not CE_HOLDING:
            ce_current_position = row['Close']
            CE_HOLDING = True
            entry_data = {
                'Entry Position': row['Close'],
                'Entry DateTime': datetime.datetime.combine(row['Date'], row['Time']),
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,
                'Symbol': "CE",
                'Entry Trend': row['Trend'],
                'Entry RSI': row['RSI_EMA'],
                'Exit Trend': None
            }
            CE_data = pd.concat([CE_data, pd.DataFrame([entry_data])], ignore_index=True)
            return
    if CE_EXIT(row):
        if CE_HOLDING:
            current_pnl = (row['Close'] - ce_current_position)
            ce_inisal_amount += current_pnl
            CE_HOLDING = False
            CE_data.loc[CE_data.index[-1], 'Exit DateTime'] = datetime.datetime.combine(row['Date'], row['Time'])
            CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
            CE_data.loc[CE_data.index[-1], 'Exit Position'] = row['Close']
            CE_data.loc[CE_data.index[-1], 'Exit Trend'] = row['Trend']
            return

# ----------------------------------------------[ PE ]----------------------------------------------------
    if PE_ENTRY(row):
        if not PE_HOLDING:
            pe_current_position = row['Close']
            PE_HOLDING = True
            entry_data = {
                'Entry Position': row['Close'],
                'Entry DateTime': datetime.datetime.combine(row['Date'], row['Time']),
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,                
                'Symbol': "PE",
                'Entry Trend': row['Trend'],
                'Entry RSI': row['RSI_EMA'],
                'Exit Trend': None  }
            PE_data = pd.concat([PE_data, pd.DataFrame([entry_data])], ignore_index=True)
            return
            
    if PE_EXIT(row):
        if PE_HOLDING:
            current_pnl = (pe_current_position - row['Close']) 
            pe_inisal_amount += current_pnl
            PE_HOLDING = False
            # Update the PE_data DataFrame with exit information
            PE_data.loc[PE_data.index[-1], 'Exit DateTime'] = datetime.datetime.combine(row['Date'], row['Time'])
            PE_data.loc[PE_data.index[-1], 'Current P&L'] = current_pnl
            PE_data.loc[PE_data.index[-1], 'Exit Position'] = row['Close']
            PE_data.loc[PE_data.index[-1], 'Exit Trend'] = row['Trend']
            return
            
data.apply(Main_Stetagy_Run, axis=1)

# --------------------------------------------------------------------------------------------------

if CE_HOLDING:
    last_price_today = round(data.iloc[-1]['Close'],2)
    current_pnl  = last_price_today-ce_current_position
    ce_inisal_amount = ce_inisal_amount + current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit DateTime'] =  f"{data.iloc[-1]['Date']} {data.iloc[-1]['Time']}"
    CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit Position'] = last_price_today

if PE_HOLDING:
    last_price_today = round(data.iloc[-1]['Close'],2)
    current_pnl  = last_price_today - pe_current_position
    pe_inisal_amount = pe_inisal_amount + current_pnl
    PE_data.loc[PE_data.index[-1], 'Exit DateTime'] =  f"{data.iloc[-1]['Date']} {data.iloc[-1]['Time']}"
    PE_data.loc[PE_data.index[-1], 'Current P&L'] = current_pnl
    PE_data.loc[PE_data.index[-1], 'Exit Position'] = last_price_today


ce_positive_pnl = CE_data[CE_data['Current P&L'] > 0]
ce_negative_pnl = CE_data[CE_data['Current P&L'] < 0]
ce_sum_positive_pnl = ce_positive_pnl['Current P&L'].sum()
ce_sum_negative_pnl = abs(ce_negative_pnl['Current P&L']).sum()



pe_positive_pnl = PE_data[PE_data['Current P&L'] > 0]
pe_negative_pnl = PE_data[PE_data['Current P&L'] < 0]
pe_sum_positive_pnl = pe_positive_pnl['Current P&L'].sum()
pe_sum_negative_pnl = abs(pe_negative_pnl['Current P&L']).sum()


print("TOTAL P&L CE: ",ce_inisal_amount ,"|| TOTAL P&L PE: ",pe_inisal_amount)
print(f" CE Positive P&L: {ce_sum_positive_pnl} || CE Negative P&L: {ce_sum_negative_pnl}")
print(f" PE Positive P&L: {pe_sum_positive_pnl} || PE Negative P&L: {pe_sum_negative_pnl}")
print("TOATL TRED CE : ",CE_data.shape[0] , "|| TOATL TRED PE : ",PE_data.shape[0])

In [ ]:
# CE_data['Entry DateTime'] = pd.to_datetime(CE_data['Entry DateTime'])
# CE_data['Exit DateTime'] = pd.to_datetime(CE_data['Exit DateTime'])
# CE_data['Tred Time'] = (CE_data['Exit DateTime'] - CE_data['Entry DateTime']).dt.total_seconds() / 60
# CE_data

In [ ]:
# PE_data['Entry DateTime'] = pd.to_datetime(PE_data['Entry DateTime'])
# PE_data['Exit DateTime'] = pd.to_datetime(PE_data['Exit DateTime'])
# PE_data['Tred Time'] = (PE_data['Exit DateTime'] - CE_data['Entry DateTime']).dt.total_seconds() / 60
# PE_data